In [ ]:
pip install chainladder

  Using cached chainladder-0.8.23-py3-none-any.whl.metadata (3.2 kB)
  Using cached numba-0.60.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached sparse-0.15.4-py2.py3-none-any.whl.metadata (4.5 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached llvmlite-0.43.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
Using cached chainladder-0.8.23-py3-none-any.whl (1.3 MB)
Using cached numba-0.60.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 MB)
Using cached sparse-0.15.4-py2.py3-none-any.whl (237 kB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
Using cached llvmlite-0.43.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.9 MB)


In [2]:
import chainladder as cl
import numpy as np
import pandas as pd

In [3]:
xyz = cl.load_sample("xyz")
print(xyz["Paid"])

         12       24       36       48       60       72       84       96       108      120      132
1998     NaN      NaN   6309.0   8521.0  10082.0  11620.0  13242.0  14419.0  15311.0  15764.0  15822.0
1999     NaN   4666.0   9861.0  13971.0  18127.0  22032.0  23511.0  24146.0  24592.0  24817.0      NaN
2000  1302.0   6513.0  12139.0  17828.0  24030.0  28853.0  33222.0  35902.0  36782.0      NaN      NaN
2001  1539.0   5952.0  12319.0  18609.0  24387.0  31090.0  37070.0  38519.0      NaN      NaN      NaN
2002  2318.0   7932.0  13822.0  22095.0  31945.0  40629.0  44437.0      NaN      NaN      NaN      NaN
2003  1743.0   6240.0  12683.0  22892.0  34505.0  39320.0      NaN      NaN      NaN      NaN      NaN
2004  2221.0   9898.0  25950.0  43439.0  52811.0      NaN      NaN      NaN      NaN      NaN      NaN
2005  3043.0  12219.0  27073.0  40026.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN
2006  3531.0  11778.0  22819.0      NaN      NaN      NaN      NaN      N

In [5]:
print(cl.Development(n_periods=3).fit_transform(xyz["Paid"]).ldf_)
print(cl.Development(n_periods=5).fit_transform(xyz["Paid"]).ldf_)
print(cl.Development(n_periods=-1).fit_transform(xyz["Paid"]).ldf_)
development_estimator = cl.Development(n_periods=-1).fit_transform(xyz["Paid"])
print(development_estimator.ldf_)
print(development_estimator.cdf_)

          12-24     24-36    36-48    48-60     60-72     72-84     84-96    96-108   108-120   120-132
(All)  3.549639  2.237557  1.61868  1.34871  1.222398  1.140765  1.050787  1.029785  1.016991  1.003679
          12-24     24-36     36-48     48-60     60-72     72-84   84-96    96-108   108-120   120-132
(All)  3.696595  2.129257  1.601152  1.342896  1.217529  1.128576  1.0555  1.029785  1.016991  1.003679
          12-24     24-36     36-48     48-60    60-72     72-84   84-96    96-108   108-120   120-132
(All)  3.765578  2.096169  1.559481  1.329354  1.21295  1.128576  1.0555  1.029785  1.016991  1.003679
          12-24     24-36     36-48     48-60    60-72     72-84   84-96    96-108   108-120   120-132
(All)  3.765578  2.096169  1.559481  1.329354  1.21295  1.128576  1.0555  1.029785  1.016991  1.003679
          12-Ult    24-Ult    36-Ult    48-Ult    60-Ult    72-Ult    84-Ult    96-Ult   108-Ult   120-Ult
(All)  24.852455  6.599905  3.148556  2.018977  1.518765  1.25212

In [6]:
unpaid_estimator = cl.Chainladder().fit(development_estimator)
print(unpaid_estimator.predict(xyz["Paid"]).ibnr_)

              2261
1998           NaN
1999     91.308424
2000    762.600768
2001   1969.689315
2002   4864.677116
2003   9913.569537
2004  27396.507281
2005  40785.572480
2006  49027.905096
2007  66442.869629
2008  81313.019619


In [38]:
grid_1 = cl.GridSearch(
    estimator = cl.Pipeline(steps=[
        ('dev',cl.Development()),
        ('model',cl.Chainladder())]),
    param_grid = dict(
        dev__n_periods=[3,4,5,-1],
        dev__average=['simple','volume']),
    scoring={'IBNR': lambda x: x.named_steps.model.ibnr_.sum()},
    n_jobs = -1)
scenarios_1 = grid_1.fit(X = xyz["Paid"],sample_weight = xyz["Premium"].latest_diagonal)
print(scenarios_1.results_)

  dev__average  dev__n_periods           IBNR
0       simple               3  334350.073145
1       simple               4  322166.564146
2       simple               5  308756.745266
3       simple              -1  284720.726576
4       volume               3  308289.665846
5       volume               4  302911.156070
6       volume               5  294692.017945
7       volume              -1  282567.719265


In [39]:
grid_1

GridSearch(estimator=Pipeline(steps=[('dev', Development()),
                                     ('model', Chainladder())]),
           n_jobs=-1,
           param_grid={'dev__average': ['simple', 'volume'],
                       'dev__n_periods': [3, 4, 5, -1]},
           scoring={'IBNR': <function <lambda> at 0x7f2c0774c680>})

In [35]:
grid_2 = cl.GridSearch(
    estimator = cl.Pipeline(steps=[
        ('dev',cl.Development()),
        ('tail',cl.TailCurve()),
        ('model',cl.Benktander())]),
    param_grid = dict(
        model__n_iters=list(range(1,100,2)),
        model__apriori =[.5,.75,1]),
    scoring={'IBNR': lambda x: x.named_steps.model.ibnr_.sum()},
    n_jobs = -1)
scenarios_2 = grid_2.fit(X = xyz["Paid"],sample_weight = xyz["Premium"].latest_diagonal)
print(scenarios_2.results_)

     model__apriori  model__n_iters           IBNR
0               0.5               1  151204.465974
1               0.5               3  194348.539126
2               0.5               5  212388.626471
3               0.5               7  224115.018734
4               0.5               9  232758.028452
..              ...             ...            ...
145             1.0              91  286028.417231
146             1.0              93  286100.336924
147             1.0              95  286166.624390
148             1.0              97  286227.720771
149             1.0              99  286284.032636

[150 rows x 3 columns]


In [13]:
##Recreates the tab '2007'
xyz_07 = xyz[xyz.valuation.year < 2008]
dev_07 = cl.Development(
    n_periods = [3] * 2 + [-1] * 7
    )
cl_07 = dev_2007.fit_transform(xyz_07["Paid"])
ult_07 = cl.Chainladder().fit(cl_07)
ult_07.ultimate_

,2261
1998,"15,764"
1999,"25,320"
2000,"38,247"
2001,"42,026"
2002,"52,680"
2003,"55,311"
2004,"95,872"
2005,"94,589"
2006,"95,350"
2007,"110,104"


In [14]:
##Recreates the tab '2008 w 2007 LDF'
ult_08_w_07_LDF = ult_2007.predict(xyz["Paid"])
ult_08_w_07_LDF.ultimate_

,2261
1998,"15,822"
1999,"24,817"
2000,"37,870"
2001,"41,035"
2002,"50,378"
2003,"50,983"
2004,"84,655"
2005,"88,339"
2006,"79,726"
2007,"96,054"


In [16]:
##Recreates the tab '2008 w 2007 Sel'
import copy as cp
dev_07_sel = cp.copy(dev_07)
dev_07_sel.n_periods = dev_07_sel.n_periods + [-1]
cl_07_sel = dev_07_sel.fit_transform(xyz["Paid"])
ult_07_sel = cl.Chainladder().fit(cl_07_sel)
ult_07_sel.ultimate_

,2261
1998,"15,822"
1999,"24,908"
2000,"37,545"
2001,"40,489"
2002,"49,302"
2003,"49,234"
2004,"80,208"
2005,"80,812"
2006,"71,847"
2007,"83,590"


In [17]:
##Recreates the tab '1997'
cl_08 = cl.Development(
    n_periods = -1
    ).fit_transform(xyz["Paid"])
ult_08 = cl.Chainladder().fit(cl_08)
ult_08.ultimate_

,2261
1998,"15,822"
1999,"24,908"
2000,"37,545"
2001,"40,489"
2002,"49,302"
2003,"49,234"
2004,"80,208"
2005,"80,812"
2006,"71,847"
2007,"78,308"
